# Étape 1 : Télécharger les données de mortalité
Nous devons d'abord télécharger les données de mortalité pour l'Angleterre et le Pays de Galles (GBRTENW) depuis la Human Mortality Database.

In [ ]:
install.packages("StMoMo")
install.packages("demography")



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘dotCall64’, ‘qvcalc’, ‘relimp’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘plyr’, ‘spam’, ‘maps’, ‘gnm’, ‘forecast’, ‘rootSolve’, ‘fanplot’, ‘reshape2’, ‘fields’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘abind’, ‘rbibutils’, ‘checkmate’, ‘RCurl’, ‘magic’, ‘lpSolve’, ‘linprog’, ‘RcppProgress’, ‘gss’, ‘stabledist’, ‘gbutils’, ‘Rdpack’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘mvtnorm’, ‘fds’, ‘deSolve’, ‘geometry’, ‘proxy’, ‘mnormt’, ‘GPArotation’, ‘fBasics’, ‘timeSeries’, ‘fastICA’, ‘cvar’, ‘Hmisc’, ‘pracma’, ‘numDeriv’, ‘RcppEigen’, ‘Lmoments’, ‘locfit’, ‘ash’, ‘FNN’, ‘kernlab’, ‘mclust’, ‘multicool’, ‘MatrixModels’, ‘snakecase’, ‘sde’, ‘pcaPP’, ‘fda’, ‘pdfCluster’, ‘ecp’, ‘e1071’, ‘psych’, ‘fGarch’, ‘vars

In [ ]:
install.packages("dplyr")
install.packages("tidyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(StMoMo)
library(demography)

Loading required package: gnm

Loading required package: forecast

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



ERROR: Error in library(demography): there is no package called ‘demography’


In [ ]:
england <- hmd.mx(country = "GBRTENW", username = "islem.maiti@esprit.tn", password = "Islem12345..")

mortality_data <- extract.ages(england, 50)

attach(england)

In [ ]:
class(england)


In [ ]:
summary(england)


In [ ]:
head(england)

In [ ]:
par(mfrow=c(1,3))
plot(england,series="male",datatype="rate", main="Male rates")
plot(england,series="female",datatype="rate", main="Female rates")
plot(england,series="total",datatype="rate", main="Total rates")

In [ ]:
par(mfrow=c(1,3))
plot(england,series="male",datatype="rate",plot.type="time", main="Male rates",xlab="Years")
plot(england,series="female",datatype="rate",plot.type="time", main="Female rates",xlab="Years")
plot(england,series="total",datatype="rate",plot.type="time", main="Total rates",xlab="Years")


#Phase 2 : VAP de la rente viagère anticipée et du capital décès




  * Les taux de mortalité utilisés (de l'année 2018) jouent un rôle crucial dans le calcul de la VAP. Des taux de mortalité plus faibles (indiquant une plus grande longévité) augmenteraient la VAP de la rente viagère et pourraient réduire celle du capital décès si la période d'attente avant décès s'allonge.


 *  **Impact des Taux d'Intérêt :**

  Le taux d'intérêt annuel de 3% utilisé dans les calculs est un facteur important. Des variations dans ce taux affecteraient significativement les VAP. Un taux plus élevé diminuerait les VAP en actualisant davantage les paiements futurs, tandis qu'un taux plus bas augmenterait les VAP.



---



  * Pour calculer la valeur actuelle probable (VAP) de la rente viagère anticipée et du capital décès en utilisant les taux de mortalité de l'année 2018, nous avons structurer votre code comme suit:
  
       *   Extraction des taux de mortalité de 2018.

       *   Calcul de la VAP de la rente viagère anticipée.

       *   Calcul de la VAP du capital décès.









In [ ]:
male_mx <- mortality_data$rate$male


# Convertir les taux de mortalité en une matrice avec les bonnes dimensions
# Dans ce cas, il n'y a qu'une seule ligne, donc cela doit être adapté à vos âges spécifiques
if (dim(male_mx)[1] == 1 && length(male_mx) == length(year)) {
  mx <- matrix(rep(male_mx, each = length(age)), nrow = length(age), ncol = length(year))
}

# Créer une nouvelle liste de taux de mortalité mise à jour pour `mortality_data`
new_mortality_data <- mortality_data
new_mortality_data$rate$male <- mx

# Mettre à jour les âges et les années dans l'objet
new_mortality_data$age <- age
new_mortality_data$year <- year

mort_2018 <- mortality_data$rate[mortality_data$year == 2018]
mort_2018$rate <- as.numeric(mort_2018$rate)



# Par exemple, si les données de taux de mortalité sont dans un composant nommé 'rate' et les années dans 'year'
mort_rates <- mortality_data$rate
mort_years <- mortality_data$year


# Extraire les taux de mortalité pour l'année 2018
mort_2018 <- mort_rates[mort_years == 2018]
# Extraction des taux de mortalité pour l'année 2018
mort_2018 <- new_mortality_data$rate$male[, which(new_mortality_data$year == 2018)]

mort_2018 <- as.numeric(mort_2018)


# Calcul des probabilités de survie annuelle
px <- 1 - mort_2018


# Calcul des probabilités de survie cumulée
survival_probs <- cumprod(px)

# Paramètres de la rente
rente_mensuelle <- 2000
taux_interet_annuel <- 0.03  # Exemple de taux d'intérêt annuel
taux_interet_mensuel <- taux_interet_annuel / 12

# Calcul de la VAP de la rente viagère anticipée
VAP_rente <- sum(rente_mensuelle * survival_probs / (1 + taux_interet_mensuel)^(0:(20*12-1)))

# Probabilité de décès annuelle
qx <- mort_2018


# Capital décès
capital_deces <- 10000

# Calcul de la VAP du capital décès
VAP_deces <- sum(capital_deces * qx * survival_probs / (1 + taux_interet_annuel)^(1:20))


In [ ]:
# Afficher les résultats
print(paste("VAP de la rente viagère anticipée :", round(VAP_rente, 2)))
print(paste("VAP du capital décès :", round(VAP_deces, 2)))

*Interprétation :*

**VAP de la Rente Viagère Anticipée**

Résultat: VAP de la rente viagère anticipée : 139332.93

Le calcul a été fait en considérant une rente mensuelle de 2000 et un taux d'intérêt annuel de 3%, converti en taux mensuel.

Montant Élevé : Une VAP de 139332.93 indique que les paiements de rente mensuelle de 2000 sont assez importants et que la personne est censée vivre encore plusieurs années pour que la somme des paiements atteigne cette valeur.
Impact de la Mortalité : La VAP est directement influencée par les taux de mortalité. Si les taux de mortalité étaient plus bas (indiquant une espérance de vie plus longue), la VAP serait encore plus élevée car les paiements seraient faits sur une période plus longue.


**VAP du Capital Décès**


Résultat: VAP du capital décès : 6949.67


Le calcul a été effectué en utilisant un capital décès de 10000 et un taux d'intérêt annuel de 3%.


Montant Relativement Bas : Une VAP de 6949.67 indique que, bien que le capital décès soit de 10000, la probabilité de décès chaque année réduit la valeur actuelle de ce paiement.
Probabilité de Décès : Le montant plus bas reflète les probabilités de décès annuelles qui, bien que non négligeables, ne sont pas suffisamment élevées pour que le paiement complet de 10000 soit fréquent. Les taux de mortalité de 2018 impliquent que la plupart des assurés vivront plusieurs années, réduisant ainsi la probabilité de paiement immédiat du capital décès.


# Phase 3 :Estimation des paramètres d’un modèle de Lee-Carter


\
Pour estimer les paramètres du modèle de Lee-Carter, nous utilisons les données de mortalité historiques de l'Angleterre. Le modèle de Lee-Carter est un modèle bien connu pour la modélisation et la projection des taux de mortalité.

##  Chargement des données et préparation initiale

In [ ]:

eng.StMoMoData<-StMoMoData(data=england, series = "total",type="central")

eng.StMoMoData.Ini<-central2initial(eng.StMoMoData)
eng.StMoMoData.Ini



##  Choix de la plage d’âge et de la période pour la calibration

Nous avons choisi de calibrer les données pour les âges de 50 à 70 ans. Cette plage d'âge est pertinente car elle couvre une période critique de la mortalité adulte où les variations et tendances sont significatives. Les années choisies correspondent à celles disponibles dans les données historiques.


In [ ]:
ages.fit = 50:70

### Génération de la matrice de pondération
Pour la calibration, nous générons une matrice de pondération qui aide à lisser les données et à réduire l'impact des valeurs extrême

In [ ]:
wxt <- genWeightMat(ages = ages.fit, years = eng.StMoMoData.Ini$years,clip = 3)

### Ajustement du modèle de Lee-Carter
Nous utilisons le modèle de Lee-Carter avec un lien logarithmique pour ajuster les données. Le modèle est ensuite ajusté aux données historiques préparées



---



Les paramètres estimés du modèle de Lee-Carter comprennent les vecteurs ax, bx et la série temporelle kt. Ces paramètres sont essentiels pour comprendre la structure de la mortalité et ses tendances au fil du temps.


*   𝛼x :  représente le niveau moyen des taux de mortalité à chaque âge.


*   bx : mesure la sensibilité des taux de mortalité à chaque âge par rapport aux changements temporels.
*   kt :  est une série temporelle représentant l'évolution des taux de mortalité au fil du temps.




In [ ]:
LC <- lc(link = "log")
LCfit <- fit(LC, data = eng.StMoMoData.Ini, ages.fit = ages.fit, wxt = wxt)
LCfit


In [ ]:
plot(LCfit)


Commentaires:

  * 𝛼𝑥
  ​
    (Niveau de mortalité par âge):
  On observe que
  𝛼𝑥
  augmente avec l'âge, ce qui est conforme à l'augmentation naturelle de la mortalité avec l'âge.

  * 𝛽𝑥(Sensibilité par âge):
  La valeur de
  𝛽
  𝑥
  diminue avec l'âge, indiquant que les âges plus jeunes sont plus sensibles aux changements temporels de la mortalité par rapport aux âges plus avancés.

  * 𝑘𝑡 (Indice temporel):
  L'indice temporel
  𝑘
  𝑡
  montre une tendance générale à la baisse, reflétant les améliorations dans la mortalité au fil du temps. Cependant, certaines fluctuations sont visibles, indiquant des périodes spécifiques de variation plus élevée ou plus faible.

#### Résidus :
 L'analyse des résidus permet de vérifier si le modèle de Lee-Carter capture correctement les tendances sous-jacentes des données de mortalité. Les résidus devraient idéalement être aléatoires et ne pas montrer de patterns systématiques, indiquant un bon ajustement du modèle.

In [ ]:
residuals_lc <- residuals(LCfit)
plot(residuals_lc)


Commentaires:

 * Résidus par âge:

Les résidus sont globalement distribués autour de zéro, mais il y a une certaine hétéroscédasticité visible, avec une variabilité plus grande aux âges extrêmes de la plage (50 et 70 ans).

* Résidus par année calendaire:

Les résidus montrent une distribution assez homogène autour de zéro, bien qu'il y ait quelques périodes où les résidus semblent plus dispersés, suggérant des changements de mortalité non capturés par le modèle pour ces périodes.

 * Résidus par année de naissance:

Comme pour les autres graphiques de résidus, ils sont globalement autour de zéro, mais une plus grande variabilité est observée pour les cohortes plus anciennes.

# Projection des taux de mortalité

La projection permet de visualiser comment les taux de mortalité pourraient évoluer à l'avenir en fonction des tendances passées capturées par le modèle.


Nous projetons les taux de mortalité pour les 25 prochaines années en utilisant les paramètres estimés du modèle de Lee-Carter.

In [ ]:
LcFore=forecast(LCfit, h = 25 ,method=mean)
EngForcast <- forecast(LCfit , h = 25, method=mean)
plot(EngForcast, only.kt=TRUE)



**Quantité Correspondante à la Projection Centrale**

Taux de Mortalité Moyens Futurs :

Les valeurs projetées de
𝜅𝑡
sont utilisées pour estimer les taux de mortalité futurs. En combinant ces valeurs avec les paramètres
𝛼𝑥 et 𝛽𝑥
, on obtient les taux de mortalité projetés pour chaque âge et chaque année.
La projection centrale des taux de mortalité futurs est donc représentée par les valeurs attendues des taux de mortalité, dérivées des valeurs centrales de
𝜅𝑡.

**Interprétation de la Figure**

Incertitude des Projections :

Les bandes grises autour de la trajectoire centrale de
𝜅𝑡
indiquent l'incertitude des projections. Plus les bandes sont larges, plus l'incertitude est grande.

La ligne noire centrale est la projection centrale, représentant la meilleure estimation des taux de mortalité futurs selon le modèle.

# les log taux de mortalités projetés pour la cohortes

Pour afficher les log taux de mortalité projetés pour la cohorte née en 1960 à partir de 2010, et calculer l'espérance de vie résiduelle de cette cohorte, suivez ces étapes :

* Extraction des Taux de Mortalité pour la Cohorte née en 1960
* Combinaison des Taux de Mortalité Historiques et Projetés
* Affichage des Log Taux de Mortalité
* Calcul de l'Espérance de Vie Résiduelle

In [ ]:

# Extraire les taux de mortalité pour la cohorte née en 1960
cohort_year <- 1960
cohort_ages <- 50:75  # Âges de la cohorte en 2010 à 2035
historical_rates <- extractCohort(fitted(LCfit, type = "rates"), cohort = cohort_year)
forecasted_rates <- extractCohort(LcFore$rates, cohort = cohort_year)

# Combiner les taux de mortalité historiques et projetés
combined_rates <- c(historical_rates, forecasted_rates)
print(length(combined_rates))  # Devrait être 76 pour 50-125 ans (environ)


combined_rates <- c(historical_rates, forecasted_rates)

combined_ages <- cohort_ages[1]:(cohort_ages[1] + length(combined_rates) - 1)

plot(combined_ages, log(combined_rates), type = "l",
     xlim = c(50, 100), xlab = "Âge", ylab = "Log des taux de mortalité",
     main = "Log des taux de mortalité projetés pour la cohorte née en 1960")
lines(combined_ages, log(combined_rates), col = "red")




### Calculer l'espérance de vie résiduelle à partir de 2010

In [ ]:
qx_proj <- LcFore$rates
lx <- matrix(NA, nrow = nrow(qx_proj), ncol = ncol(qx_proj))
lx[, 1] <- 1  # lx à l'âge 0 est 1

for (i in 2:ncol(qx_proj)) {
  lx[, i] <- lx[, i - 1] * (1 - qx_proj[, i - 1])
}



# Calculer l'espérance de vie résiduelle
qx_proj <- combined_rates

lx <- numeric(length(qx_proj))
lx[1] <- 1  # lx à l'âge initial est 1


for (i in 2:length(qx_proj)) {
  lx[i] <- lx[i - 1] * (1 - qx_proj[i - 1])
}

# Calculer l'espérance de vie résiduelle
ex_proj <- sum(lx) / lx[1]
print(paste("Espérance de vie résiduelle à partir de 2010:", round(ex_proj, 2)))


**Interprétation :**

 Le résultat obtenu, 20 ans, représente l'espérance de vie résiduelle pour la cohorte née en 1960 à partir de l'année 2010. Cela signifie que, selon les taux de mortalité projetés, une personne de cette cohorte peut s'attendre à vivre environ 20 années supplémentaires à partir de 2010.

#### Comparaison à avec l’espérance de vie périodique en 2010.

>  Extraire les taux de mortalité pour l'année 2010



In [ ]:
mortality_2010 <- england$rate$male[, which(mortality_data$year == 2010)]

mortality_2010 <- as.numeric(mortality_2010)

px_2010 <- 1 - mortality_2010

# Calculer lx, la proportion de survivants à chaque âge
lx_2010 <- cumprod(c(1, px_2010))

# Calculer l'espérance de vie périodique en 2010
ex_2010 <- sum(lx_2010) / lx_2010[1]
print(paste("Espérance de vie périodique en 2010:", round(ex_2010, 2)))

# Comparaison de l'espérance de vie résiduelle et de l'espérance de vie périodique
## Résultats obtenus :
Espérance de vie résiduelle pour la cohorte née en 1960 à partir de 2010 : 20,01 ans

Espérance de vie périodique en 2010 : 78,85 ans


### Comparaison et interprétation :

Nature des mesures :


* Espérance de vie résiduelle est spécifique à une cohorte (les personnes nées en 1960) et prend en compte les changements futurs dans les taux de mortalité. C'est une mesure dynamique qui intègre des projections sur l'évolution de la mortalité.

* Espérance de vie périodique est une mesure statique basée sur les taux de mortalité d'une année particulière (2010). Elle suppose que ces taux restent constants tout au long de la vie des individus.

### Valeurs obtenues :

L'espérance de vie résiduelle de 20,01 ans pour la cohorte née en 1960 à partir de 2010 signifie que, selon les projections, une personne de cette cohorte vivant en 2010 peut s'attendre à vivre environ 20 ans de plus.
L'espérance de vie périodique de 78,85 ans en 2010 représente la durée de vie moyenne attendue pour une personne née en 2010 si les taux de mortalité de cette année restaient constants toute sa vie.

#   VAP des deux contrats en utilisant les taux projetés.

Pour calculer la Valeur Actuarielle Présente (VAP) des deux contrats en utilisant les taux de mortalité projetés, nous avons défini deux fonctions R :
* calc_vap_rente pour la rente viagère anticipée
* calc_vap_deces pour le capital décès

In [ ]:
# Fonction pour calculer la VAP de la rente viagère anticipée
calc_vap_rente <- function(rente_mensuelle, taux_mortalite, taux_interet_annuel) {
  # Calculer le taux d'intérêt mensuel
  taux_interet_mensuel <- taux_interet_annuel / 12

  # Calculer les probabilités de survie annuelle
  px <- 1 - taux_mortalite

  # Calculer les probabilités de survie cumulée
  survival_probs <- cumprod(px)

  # Calculer la VAP de la rente viagère anticipée
  VAP_rente <- sum(rente_mensuelle * survival_probs / (1 + taux_interet_mensuel)^(0:(length(survival_probs) * 12 - 1)))

  return(VAP_rente)
}

# Exemple d'utilisation
rente_mensuelle <- 2000
taux_interet_annuel <- 0.03
taux_mortalite_proj <- combined_rates  # Taux de mortalité projetés

VAP_rente <- calc_vap_rente(rente_mensuelle, taux_mortalite_proj, taux_interet_annuel)
print(paste("VAP de la rente viagère anticipée :", round(VAP_rente, 2)))



  # Fonction pour calculer la VAP du capital décès
  calc_vap_deces <- function(capital_deces, taux_mortalite, taux_interet_annuel) {
    # Calculer les probabilités de survie annuelle
    px <- 1 - taux_mortalite

    # Calculer les probabilités de survie cumulée
    survival_probs <- cumprod(px)

    # Calculer la VAP du capital décès
    VAP_deces <- sum(capital_deces * taux_mortalite * survival_probs / (1 + taux_interet_annuel)^(1:length(taux_mortalite)))

    return(VAP_deces)
  }

capital_deces <- 10000


VAP_deces <- calc_vap_deces(capital_deces, taux_mortalite_proj, taux_interet_annuel)
print(paste("VAP du capital décès :", round(VAP_deces, 2)))



# **Analyse des VAP**


**Rente viagère anticipée :**


* La VAP de 354,524.38 représente la valeur actuelle de tous les paiements mensuels de 2,000 unités que l'assuré recevrait tout au long de sa vie, actualisés au taux d'intérêt annuel de 3%. Cette valeur est relativement élevée car elle cumule de nombreux paiements mensuels sur une longue période (environ 20 ans d'espérance de vie résiduelle).

**Capital décès :**

* La VAP de 979.03 est beaucoup plus faible car elle représente un paiement unique de 10,000 unités au décès de l'assuré, actualisé par les probabilités de décès chaque année et au taux d'intérêt de 3%. Le faible montant s'explique par la combinaison de la probabilité de décès à chaque âge et l'actualisation.

### l’espérance de vie de la cohorte et les VAP des produits

In [ ]:
# Définir les paramètres du choc de mortalité
taux_augmentation <- 0.20
annees_changement <- 2020:2025
years_proj <- LcFore$years

# Fonction pour recalculer l'espérance de vie résiduelle avec les nouveaux taux de mortalité projetés
recalculer_esp_vie_residuelle <- function(qx_proj, taux_augmentation, annees_changement, annees_proj) {
  indices <- which(annees_proj %in% annees_changement)
  qx_proj[indices] <- qx_proj[indices] * (1 + taux_augmentation)
  lx <- numeric(length(qx_proj))
  lx[1] <- 1  # lx à l'âge initial est 1
  for (i in 2:length(qx_proj)) {
    lx[i] <- lx[i - 1] * (1 - qx_proj[i - 1])
  }
  ex_proj <- sum(lx) / lx[1]
  return(ex_proj)
}

# Recalcul de l'espérance de vie résiduelle avec choc de mortalité
qx_proj <- combined_rates
annees_proj <- years_proj  # Assurez-vous que cette variable contient les années projetées
ex_proj_choc <- recalculer_esp_vie_residuelle(qx_proj, taux_augmentation, annees_changement, annees_proj)
print(paste("Nouvelle espérance de vie résiduelle avec choc de mortalité à partir de 2020:", round(ex_proj_choc, 2)))

# Fonction pour calculer la VAP de la rente viagère anticipée
calc_vap_rente <- function(rente_mensuelle, taux_mortalite, taux_interet_annuel) {
  # Calculer le taux d'intérêt mensuel
  taux_interet_mensuel <- (1 + taux_interet_annuel)^(1/12) - 1

  # Calculer les probabilités de survie annuelle
  px <- 1 - taux_mortalite

  # Calculer les probabilités de survie cumulée
  survival_probs <- cumprod(px)

  # Calculer la VAP de la rente viagère anticipée
  VAP_rente <- sum(rente_mensuelle * survival_probs / (1 + taux_interet_mensuel)^(0:(length(survival_probs) - 1)))

  return(VAP_rente)
}

# Exemple d'utilisation avec les nouveaux taux de mortalité
rente_mensuelle <- 2000
taux_interet_annuel <- 0.03
VAP_rente_choc <- calc_vap_rente(rente_mensuelle, qx_proj, taux_interet_annuel)
print(paste("VAP de la rente viagère anticipée avec choc de mortalité:", round(VAP_rente_choc, 2)))

# Fonction pour calculer la VAP du capital décès
calc_vap_deces <- function(capital_deces, taux_mortalite, taux_interet_annuel) {
  # Calculer les probabilités de survie annuelle
  px <- 1 - taux_mortalite

  # Calculer les probabilités de survie cumulée
  survival_probs <- cumprod(px)

  # Calculer la VAP du capital décès
  VAP_deces <- sum(capital_deces * taux_mortalite * c(1, head(survival_probs, -1)) / (1 + taux_interet_annuel)^(1:length(taux_mortalite)))

  return(VAP_deces)
}

# Exemple d'utilisation avec les nouveaux taux de mortalité
capital_deces <- 10000
VAP_deces_choc <- calc_vap_deces(capital_deces, qx_proj, taux_interet_annuel)


print(paste("Nouvelle espérance de vie résiduelle avec choc de mortalité", round(ex_proj_choc, 2)))
print(paste("VAP du capital décès avec choc de mortalité:", round(VAP_deces_choc, 2)))


### * Nouvelle espérance de vie résiduelle avec choc de mortalité à partir de 2020: 19.98

Cette valeur indique que même après l'application du choc de mortalité, l'espérance de vie résiduelle de la cohorte reste relativement stable par rapport à sa valeur précédente. Cela suggère que bien que les taux de mortalité aient augmenté de 20% entre 2020 et 2025, l'impact sur l'espérance de vie résiduelle est limité, ce qui peut être interprété comme une résilience de la population face à ce choc de mortalité.

### * VAP de la rente viagère anticipée avec choc de mortalité:38793.65

Cette valeur indique que malgré le choc de mortalité, la valeur actuelle probable des paiements mensuels de la rente viagère anticipée reste relativement élevée. Cela suggère que même avec une augmentation des taux de mortalité, la rente viagère demeure un produit d'assurance attrayant en raison de la sécurité financière qu'elle offre sur une longue période.

#### * VAP du capital décès avec choc de mortalité: 988.52

Cette valeur suggère que même après le choc de mortalité, la valeur actuelle probable du capital décès reste relativement basse en raison de sa nature de paiement unique. Cela signifie que bien que le choc de mortalité puisse augmenter les risques de décès, l'impact financier immédiat sur le capital décès demeure limité.